In [ ]:
#pip install --upgrade ipykernel

In [ ]:
#pip install biterm

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

texts = open('./data_test/reuters.titles').read().splitlines()[:50]
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(texts).toarray()

In [2]:
texts

['UK: Prince Charles spearheads British royal revolution.',
 'GERMANY: Historic Dresden church rising from WW2 ashes.',
 "INDIA: Mother Teresa's condition said still unstable.",
 'UK: Palace warns British weekly over Charles pictures.',
 'INDIA: Mother Teresa, slightly stronger, blesses nuns.',
 "INDIA: Mother Teresa's condition unchanged, thousands pray.",
 'INDIA: Mother Teresa shows signs of strength, blesses nuns.',
 "INDIA: Mother Teresa's condition improves, many pray.",
 'INDIA: Mother Teresa improves, nuns pray for "miracle".',
 'UK: Charles under fire over prospect of Queen Camilla.',
 'UK: Britain tells Charles to forget Camilla.',
 "COTE D'IVOIRE: FEATURE - Quiet homecoming for reprieved Ivory Coast maid.",
 'INDIA: Mother Teresa ("I want to go home") sits and prays.',
 'INDIA: Mother Teresa nears end of crisis, nuns rejoice.',
 'UK: Prosaic end for marriage of Charles and Diana.',
 'UK: No respite for British royals despite divorce.',
 "UK: Camilla, love of Charles' life, a

In [3]:
from biterm.utility import vec_to_biterms

vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X)

In [4]:
vocab

array(['74', 'actor', 'actress', 'admits', 'africa', 'airplay', 'amid',
       'angel', 'angered', 'appeals', 'archbishop', 'architect',
       'argentina', 'ashes', 'attacks', 'backstage', 'barons', 'battered',
       'bernardin', 'bishop', 'blesses', 'blond', 'born', 'brazil',
       'britain', 'british', 'bucharest', 'business', 'calls', 'camilla',
       'cancer', 'cardinal', 'catholic', 'charles', 'child', 'chile',
       'chilean', 'church', 'coast', 'condition', 'controversy', 'cool',
       'cote', 'crisis', 'daughter', 'day', 'debate', 'demanding',
       'despite', 'diana', 'dies', 'discussed', 'disney', 'divorce',
       'draws', 'dresden', 'economist', 'elizabeth', 'emmys', 'end', 'ex',
       'eyes', 'family', 'feature', 'felipe', 'figure', 'finally',
       'forget', 'france', 'funeral', 'future', 'german', 'germany',
       'greece', 'greek', 'health', 'herrera', 'hints', 'historic',
       'hold', 'holds', 'home', 'homecoming', 'honors', 'house',
       'hungary', 'impr

In [ ]:
from biterm.cbtm import oBTM

btm = oBTM(num_topics=20, V=vocab)
topics = btm.fit_transform(biterms, iterations=100)

In [ ]:
import numpy as np
import pyLDAvis
from biterm.cbtm import oBTM 
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary # helper functions

if __name__ == "__main__":

    texts = open('./data_test/reuters.titles').read().splitlines()

    # vectorize texts
    vec = CountVectorizer(stop_words='english')
    X = vec.fit_transform(texts).toarray()

    # get vocabulary
    vocab = np.array(vec.get_feature_names())

    # get biterms
    biterms = vec_to_biterms(X)

    # create btm
    btm = oBTM(num_topics=20, V=vocab)

    print("\n\n Train Online BTM ..")
    for i in range(0, len(biterms), 100): # prozess chunk of 200 texts
        biterms_chunk = biterms[i:i + 100]
        btm.fit(biterms_chunk, iterations=50)
    topics = btm.transform(biterms)

    print("\n\n Visualize Topics ..")
    vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X, axis=1), vocab, np.sum(X, axis=0))
    pyLDAvis.save_html(vis, 'online_btm.html')
    pyLDAvis.display()

    print("\n\n Topic coherence ..")
    topic_summuary(btm.phi_wz.T, X, vocab, 10)

    print("\n\n Texts & Topics ..")
    for i in range(len(texts)):
        print("{} (topic: {})".format(texts[i], topics[i].argmax()))